In [2]:
import pandas as pd
import requests
import gzip
import shutil
from pathlib import Path
import csv

In [14]:
df = pd.read_csv('../data/processed/openfoodfacts_processed_10_000.csv')

In [15]:
df.nunique()

code                                                     7736
url                                                      7736
product_name                                             7123
quantity                                                 1651
brands                                                   2435
brands_tags                                              2072
categories                                               5586
categories_tags                                          4894
categories_en                                            4894
labels                                                   5732
labels_tags                                              5246
labels_en                                                5246
countries                                                 958
countries_tags                                            614
countries_en                                              614
ingredients_text                                         7481
ingredie

In [29]:
df[['nova_group']].value_counts().sort_index()

nova_group
1              433
2               93
3             1391
4             5876
Name: count, dtype: int64

In [19]:
df[['nutriscore_grade']].value_counts().sort_index()

nutriscore_grade
a                    799
b                    929
c                   2392
d                   2038
e                   1635
Name: count, dtype: int64

In [20]:
df[['nutriscore_score']].value_counts().sort_index()

nutriscore_score
-16                  20
-15                   3
-13                   4
-11                  10
-10                  15
-9                   14
-8                   25
-7                   16
-6                   27
-5                   45
-4                   64
-3                   88
-2                  143
-1                  209
 0                  280
 1                  318
 2                  447
 3                  429
 4                  453
 5                  428
 6                  326
 7                  291
 8                  240
 9                  167
 10                 175
 11                 318
 12                 262
 13                 265
 14                 303
 15                 228
 16                 215
 17                 202
 18                 208
 19                 224
 20                 179
 21                 161
 22                 147
 23                  99
 24                  95
 25                  93
 26                 100

In [23]:
df[['product_name','nutriscore_grade', 'nutriscore_score']].head(100)

,product_name,nutriscore_grade,nutriscore_score
0,Barre daily carotte,a,-1
1,Cold Brew Coffee,e,10
2,this is food happy banana,e,10
3,yfood Crazy Coconut,e,10
4,Hot & Savoury - Tomato & Herb,c,10
...,...,...,...
95,chocapic,c,7
96,Renutryl Booster saveur Chocolat,d,9
97,Viva chocolat 25cl,d,7
98,NESTLE LION Céréales 750g,d,14


In [28]:
df['categories'].unique()

array(["Aliments et boissons à base de végétaux, Aliments d'origine végétale, Snacks, Snacks sucrés, Barres, Compléments alimentaires, Compléments pour le Bodybuilding, Barres protéinées",
       'Getränke',
       'Fertiggerichte, Trinkmahlzeit, en:Drink-meal, Beverages', ...,
       'Plant-based foods and beverages, Beverages, Plant-based foods, Dairy substitutes, Milk substitutes, Plant-based beverages, Plant-based milk alternatives, Plant-based creams, Plant-based creams for cooking, Coconut milks and creams',
       'Plant-based foods and beverages, Plant-based foods, Breakfasts, Cereals and potatoes, Cereals and their products, Breakfast cereals, Cereals with fruits, Extruded cereals, Cereal flakes with fruits',
       'Snacks, Sweet snacks, Confectioneries, Fruit confectioneries, Candies, Liquorice candies'],
      shape=(5586,), dtype=object)